In [1]:
import re
import pandas as pd
import numpy as np

# Paquetes llamadas 
import time
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
# from selenium.webdriver.common.keys import Keys
## from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.by import By
## import urllib.request
from selenium.common.exceptions import NoSuchElementException

from selenium.webdriver.chrome.service import Service

from paquetes.funciones_partidos import extraer_info
from paquetes.funciones_arbitrales import verificar_informador
from paquetes.funciones_genericas import fila_resaltada, transformar_informe

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
## from email.mime.base import MIMEBase
## from email import encoders

## from datetime import datetime

## import ipywidgets as widgets
## from IPython.display import display

In [2]:
# Web scrapper for infinite scrolling page 
PATH = "D:/chromedriver/chromedriver.exe"
service = Service(executable_path=PATH)
OPTIONS = webdriver.ChromeOptions()
OPTIONS.add_argument("--start-maximized")

time.sleep(2)  # Allow 2 seconds for the web page to open
scroll_pause_time = 2 # You can set your own pause time. My laptop is a bit slow so I use 1 sec
# screen_height = driver.execute_script("return window.screen.height;")   # get the screen height of the web
screen_height = 12000

In [3]:
# Generamos la url y entramos a la url
url = "https://zonafcbq.basquetcatala.cat/auth"
driver = webdriver.Chrome(service=service, options=OPTIONS)

# Delete the cookies  
driver.delete_all_cookies()
driver.get(url)

time.sleep(10)

In [4]:
# OJO!!!! Nueva versión dónde se ha de introducir el usuario y contraseña:
elementos = driver.find_elements(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div/div/div[1]')

if elementos:
    # Añadir el email
    email = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div/div/div[3]/div[1]/div[1]/div[2]/input')
    
    # Wait
    driver.implicitly_wait(time_to_wait=5)

    # Type email
    email.send_keys('sergi9_14@hotmail.com')

    # Le damos a continuar en el formulario
    boton = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div/div/div[3]/div[2]/div')
    boton.click()

    # Ahora toca añadir la contraseña
    pswd = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div/div/div[3]/div[2]/div[1]/div[2]/input')
    
    # Type to pswd
    pswd.send_keys('valldaro9')

    # Damos a continuar 
    boton = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div/div/div[3]/div[3]')
    boton.click()

else:
    # Insertamos los valores de usuarios y contraseña en los casilleros
    # Get element
    email = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div[1]/div[1]/div[2]/div[1]/div[1]/div[2]/input')

    # Wait
    driver.implicitly_wait(time_to_wait=5)

    # Type email
    email.send_keys('sergi9_14@hotmail.com')

    # Lo mismo para la contraseña
    contrasenya = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div[1]/div[1]/div[2]/div[2]/div[1]/div[2]/input')

    # Wait
    driver.implicitly_wait(time_to_wait=5)

    # Type to pswd
    contrasenya.send_keys('valldaro9')

    # Le damos al botón de enviar para conectar con la web
    button = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div[1]/div[2]')
    button.click()


In [5]:
# Accedemos a la area correspondiente de las designaciones 
area_arbitro = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div[1]/div[1]/div/img')
area_arbitro.click()

In [6]:
# Accedemos a la parte dels meus partits
meus_partits = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div/div[1]/div[3]/div[2]/a')
meus_partits.click()

In [7]:
# Extraemos toda la tabla de partidos 
tabla = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div[2]/div[2]/div')
tablaPartidos = tabla.text.split("\n")
tablaPartidos = [item for item in tablaPartidos if item not in ['↓', ' Acta', ' Pàgina partit']]
tablaPartidos

# Las primeras 8 posiciones serán las columnas
columns = tablaPartidos[:7]

# El resto serán los valores que se organizan en filas de 8 elementos cada una
data = [tablaPartidos[i:i+7] for i in range(7, len(tablaPartidos), 7)]

# Crear el DataFrame
df = pd.DataFrame(data, columns=columns)

df['Num.'] = [num.strip() for num in df['Num.']]

df['urlPartido'] = 'https://www.basquetcatala.cat/partits/llistatpartits/' + df['Num.']
df


,Num.,Equip local,Equip visitant,Dia de joc,Camp,Competició,Categoria,urlPartido
0,6048,SESE - VITALDENT,TGN BÀSQUET A,08/03/2025 17:45,PAVELLO MUNICIPAL VIRREI AMAT,FASE PRÈVIA,SUPER COPA FEMENINA,https://www.basquetcatala.cat/partits/llistatp...
1,71477,COMKEDEM,CORPORACIÓ FISIOGESTIÓN CB ROSER,09/03/2025 17:00,CEM LLARS MUNDET,BCR LLIGA CATALANA 2,ESPORT ADAPTAT,https://www.basquetcatala.cat/partits/llistatp...


In [8]:
# Extraemos información de la ficha de partido 
infoPartidosFicha = []

for url in df.urlPartido:
# Realizar una solicitud GET a la página web
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, "html.parser")
        try:
            info = extraer_info(soup)
        except:
            continue
        infoPartidosFicha.append(info)

tablaFichas = pd.DataFrame(infoPartidosFicha)
tablaFichas

,local,visitante,competicion,grup,data partit,hora partit,partit,AP,AA,AA2,AN,CR,RLL,CALLER
0,SESE - VITALDENT,TGN BÀSQUET A,FASE PRÈVIA,01,08-03-2025,17:45,6048,RAQUEL GUIL GARCIA,POL MARCET CALDERS,,VICTOR GOMEZ PLAZA,SERGI RAMIREZ MITJANS,MIREIA ROBERT MATEO,MARCEL COLOME PORTA


In [9]:
# Accedemos al apartado de presupuesto 
comptabilitat = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[3]/div[2]/div/div[1]/div/div/div')
comptabilitat.click()

# Accedemos a la preliquidacion semanal
preliquidacio_setmanal = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[3]/div[3]/div/div[2]/div/div[1]/div/div/div')
preliquidacio_setmanal.click()

#### EXTRAEMOS LA TABLA ###### 
cabecera = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div[2]/div[2]/div/div[1]/div').text
cabecera = [item for item in cabecera.split("\n") if item not in ['↑', '   ']]
# cabecera.append('url')

tabla = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div[2]/div[2]/div/div[2]').text
tabla = tabla.split("\n")
matriz = np.reshape(tabla, (-1, len(cabecera)))

# urls = driver.find_element(By.XPATH, '//*[@id="root"]/div/div/div[4]/div/div[2]/div[2]/div/div[2]')

# tabla.text

# Crear el DataFrame
precio = pd.DataFrame(matriz, columns=cabecera)
precio

,Id partit,Data,Categoria,Equip local,Equip visitant,Total brut,Retenció,A cobrar
0,71477,09/03/2025,ESPORT ADAPTAT,COMKEDEM,CORPORACIÓ FISIOGESTIÓN CB ROSER,23.54 €,2.88 €,20.66 €
1,6048,08/03/2025,SUPER COPA FEMENINA,SESE - VITALDENT,TGN BÀSQUET A,29.54 €,3.96 €,25.58 €


In [10]:
driver.quit()

In [11]:
from datetime import datetime, timedelta

def get_sunday_of_current_week():
    today = datetime.now()  # Obtener la fecha y hora actuales
    days_to_sunday = 6 - today.weekday()  # Calcular los días restantes hasta el domingo
    sunday = today + timedelta(days=days_to_sunday)  # Sumar los días necesarios para llegar al domingo
    return sunday.strftime("%d/%m/%Y")  # Devolver solo la fecha

In [12]:
# def extraer_info_urls_feb(url, liga):
#    driver = webdriver.Chrome(service=service, options=OPTIONS)

    # Delete the cookies  
    #driver.delete_all_cookies()
    #driver.get(url)

    #time.sleep(10)

    #fecha_a_seleccionar = get_sunday_of_current_week()

    # Aceptamos la politica 
    #driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[3]/div/button').click()

    # Esperamos 10 segundos
    #time.sleep(10)

    # Buscar la liga en el desplegable
    #try:
    #    select = Select(driver.find_element(By.XPATH, '//*[@id="_ctl0_MainContentPlaceHolderMaster_gruposDropDownList"]'))
    #    select.select_by_visible_text(liga)
    #    print(f"Liga seleccionada: {liga}")
    #except Exception as e:
        #print(f"Error al seleccionar la liga: {e}")

    # Extraemos todas las urls necesarias 
    #urls = []
    #try:
        #enlaces = driver.find_elements(By.TAG_NAME, "a")  # Localizar todos los <a>
        #for enlace in enlaces:
            #href = enlace.get_attribute("href")  # Extraer el atributo href
            #if href:  # Comprobar que href no sea None
                #urls.append(href)
    #except Exception as e:
        #print(f"Error al extraer URLs: {e}")

    # Filtramos urls para quedarnos exclusivamente la de los partidos 
    #urls = [url for url in urls if "https://baloncestoenvivo.feb.es/Partido.aspx" in url]

    #return urls

In [13]:
# Extraemos la información de la para poder emparejar con la información de la tabla
#urls = []

#url = "https://baloncestoenvivo.feb.es/calendario/tercerafeb/3/2024"
#liga = 'Liga Regular "C-A"'
#urlsCA = extraer_info_urls_feb(url, liga)

#url = "https://baloncestoenvivo.feb.es/calendario/tercerafeb/3/2024"
#liga = 'Liga Regular "C-B"'
#urlsCB = extraer_info_urls_feb(url, liga)

#url = "https://baloncestoenvivo.feb.es/calendario/lf2/9/2024"
#liga = 'Liga Regular "A"'
#urlsA = extraer_info_urls_feb(url, liga)

#url = "https://baloncestoenvivo.feb.es/calendario/lf2/9/2024"
#liga = 'Liga Regular "B"'
#urlsB = extraer_info_urls_feb(url, liga)

#urls = [urlsCA, urlsCB, urlsA, urlsB]

In [14]:
# def extraer_info_feb(driver, url):
    
    # Delete the cookies  
    #driver.get(url)
    #time.sleep(5)
    
    # Aceptamos la politica 
    #driver.find_element(By.XPATH, '/html/body/div[3]/div/div/div[3]/div/button').click()

    # Extraemos la información necesaria
    #texto =   .find_elements(By.XPATH, '//*[@id="loader-data"]/div[1]/div[1]/div[1]/span[1]/a')
    #local = [elemento.text for elemento in texto]

    #texto = driver.find_elements(By.XPATH, '//*[@id="loader-data"]/div[1]/div[1]/div[1]/span[2]')
    #PL = [elemento.text for elemento in texto]

    #texto = driver.find_elements(By.XPATH, '//*[@id="loader-data"]/div[1]/div[1]/div[3]/span[2]/a')
    #visitante = [elemento.text for elemento in texto]

    #texto = driver.find_elements(By.XPATH, '//*[@id="loader-data"]/div[1]/div[1]/div[3]/span[1]')
    #PV = [elemento.text for elemento in texto]

    #texto = driver.find_elements(By.XPATH, '//*[@id="loader-data"]/div[3]/div[1]/span[2]')
    #fecha = [elemento.text for elemento in texto]

    #texto = driver.find_elements(By.XPATH, '//*[@id="loader-data"]/div[3]/div[2]/span[2]')
    #AP = [elemento.text for elemento in texto]

    #texto = driver.find_elements(By.XPATH, '//*[@id="loader-data"]/div[3]/div[2]/span[3]')
    #AA = [elemento.text for elemento in texto]

    #texto = driver.find_elements(By.XPATH, '//*[@id="loader-data"]/div[3]/div[3]/span[2]')
    #Pabellon = [elemento.text for elemento in texto]

    #texto = driver.find_elements(By.XPATH, '//*[@id="loader-data"]/div/div[3]/span[3]')
    #Ubicacion = [elemento.text for elemento in texto]  

    # devolvemos la informacion
    #return (local, PL, visitante, PV, fecha, AP, AA, Pabellon, Ubicacion)


In [15]:
# locales = []
# PLs = []
# visitantes = []
# PVs = []
# fechas = []
# APs = []
# AAs = []
# Pabellones = []
# Ubicaciones = []

#for url in urls: 
    # Extraemos la información de la para poder emparejar con la información de la tabla
#    driver = webdriver.Chrome(service=service, options=OPTIONS)

#    local, PL, visitante, PV, fecha, AP, AA, Pabellon, Ubicacion = extraer_info_feb(driver, url)
#    locales.extend(local), PLs.extend(PL), visitantes.extend(visitante)
#    PVs.extend(PV), fechas.extend(fecha), APs.extend(AP)
#    AAs.extend(AA), Pabellones.extend(Pabellon)
#    Ubicaciones.extend(Ubicacion)

In [16]:
# Unir DataFrames por la columna 'id'
df_temp = pd.merge(df, precio, left_on = 'Num.', right_on= 'Id partit')
df_final = pd.merge(df_temp, tablaFichas, left_on = 'Num.', right_on= 'partit', how='left')
df_final

,Num.,Equip local,Equip visitant_x,Dia de joc,Camp,Competició,Categoria,urlPartido,Id partit,Data,...,data partit,hora partit,partit,AP,AA,AA2,AN,CR,RLL,CALLER
0,6048,SESE - VITALDENT,TGN BÀSQUET A,08/03/2025 17:45,PAVELLO MUNICIPAL VIRREI AMAT,FASE PRÈVIA,SUPER COPA FEMENINA,https://www.basquetcatala.cat/partits/llistatp...,6048,08/03/2025,...,08-03-2025,17:45,6048,RAQUEL GUIL GARCIA,POL MARCET CALDERS,,VICTOR GOMEZ PLAZA,SERGI RAMIREZ MITJANS,MIREIA ROBERT MATEO,MARCEL COLOME PORTA
1,71477,COMKEDEM,CORPORACIÓ FISIOGESTIÓN CB ROSER,09/03/2025 17:00,CEM LLARS MUNDET,BCR LLIGA CATALANA 2,ESPORT ADAPTAT,https://www.basquetcatala.cat/partits/llistatp...,71477,09/03/2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
# Para convertir el MultiIndex en Index simple
df_final.columns = df_final.columns.get_level_values(0)

In [18]:
# Convertir las columnas 'fecha' y 'hora' a un formato datetime
df_final['fecha_hora'] = pd.to_datetime(df_final[' Dia de joc'], format='%d/%m/%Y %H:%M')

In [19]:
# Convertir las columnas 'fecha' y 'hora' a un formato datetime
df_final['fecha_hora'] = pd.to_datetime(df_final[' Dia de joc'], format='%d/%m/%Y %H:%M')

# Ordenar por la columna combinada 'fecha_hora'
df_sorted = df_final.sort_values(by='fecha_hora').reset_index(drop=True)

In [20]:
# Creamos la variable de precio
df_sorted["precio"] = [float(re.sub(" €", "", aC)) for aC in df_sorted[" A cobrar"]]

In [21]:
df_final = df_sorted
df_final.columns = [str.strip() for str in df_final.columns]

In [22]:
df_final

,Num.,Equip local,Equip visitant_x,Dia de joc,Camp,Competició,Categoria,urlPartido,Id partit,Data,...,partit,AP,AA,AA2,AN,CR,RLL,CALLER,fecha_hora,precio
0,6048,SESE - VITALDENT,TGN BÀSQUET A,08/03/2025 17:45,PAVELLO MUNICIPAL VIRREI AMAT,FASE PRÈVIA,SUPER COPA FEMENINA,https://www.basquetcatala.cat/partits/llistatp...,6048,08/03/2025,...,6048,RAQUEL GUIL GARCIA,POL MARCET CALDERS,,VICTOR GOMEZ PLAZA,SERGI RAMIREZ MITJANS,MIREIA ROBERT MATEO,MARCEL COLOME PORTA,2025-03-08 17:45:00,25.58
1,71477,COMKEDEM,CORPORACIÓ FISIOGESTIÓN CB ROSER,09/03/2025 17:00,CEM LLARS MUNDET,BCR LLIGA CATALANA 2,ESPORT ADAPTAT,https://www.basquetcatala.cat/partits/llistatp...,71477,09/03/2025,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025-03-09 17:00:00,20.66


In [23]:
df_final = df_final[['Num.', 'Equip local', 'Equip visitant_x', 
                     'Dia de joc', 'Camp', 'Competició', 'Categoria',
                     'AP', 'AA', 'AA2', 'AN', 'CR', 'RLL', 'CALLER', 
                     'Total brut', 'Retenció', 'A cobrar', 'precio',
                     'urlPartido']]
df_final = df_final.loc[:, ~df_final.columns.duplicated()]

In [24]:
df_final.columns = ['Num.', 'Equip local', 'Equip visitant', 
                     'Dia de joc', 'Camp', 'Competició', 'Categoria',
                     'AP', 'AA', 'AA2', 'AN', 'CR', 'RLL', 'CALLER', 
                     'Total brut', 'Retenció', 'A cobrar', 'precio',
                     'urlPartido']

In [25]:
## filtro = df_final['Categoria'].str.contains('FEB|Liga Femenina 2', case=False, regex=True)
## df_final.loc[filtro, "AP"] = "LUCAS ARIAS"
## df_final.loc[filtro, "AA"] = "NIKITA ERGIN"

In [26]:
df_final

,Num.,Equip local,Equip visitant,Dia de joc,Camp,Competició,Categoria,AP,AA,AA2,AN,CR,RLL,CALLER,Total brut,Retenció,A cobrar,precio,urlPartido
0,6048,SESE - VITALDENT,TGN BÀSQUET A,08/03/2025 17:45,PAVELLO MUNICIPAL VIRREI AMAT,FASE PRÈVIA,SUPER COPA FEMENINA,RAQUEL GUIL GARCIA,POL MARCET CALDERS,,VICTOR GOMEZ PLAZA,SERGI RAMIREZ MITJANS,MIREIA ROBERT MATEO,MARCEL COLOME PORTA,29.54 €,3.96 €,25.58 €,25.58,https://www.basquetcatala.cat/partits/llistatp...
1,71477,COMKEDEM,CORPORACIÓ FISIOGESTIÓN CB ROSER,09/03/2025 17:00,CEM LLARS MUNDET,BCR LLIGA CATALANA 2,ESPORT ADAPTAT,NaN,NaN,NaN,NaN,NaN,NaN,NaN,23.54 €,2.88 €,20.66 €,20.66,https://www.basquetcatala.cat/partits/llistatp...


In [27]:
# Detectamos si tenemos algun partido de española
filtro = df_final['Categoria'].str.contains('FEB|LIGA FEMENINA DE BALONCESTO 2|ESPORT ADAPTAT', case=False, regex=True)

# Recorrer las filas del DataFrame filtrado e ingresar los nombres de los árbitros
for idx, row in df_final.iterrows():
    if filtro[idx]:
        print(f"\nPartido: {row['Equip local']} vs {row['Equip visitant']} Categoria: {row['Categoria']}")
        
        ap = input(f"Ingrese el nombre del Árbitro Principal (AP) para el partido {row['Equip local']} vs {row['Equip visitant']}: >> ")
        aa = input(f"Ingrese el nombre del Árbitro Asistente (AA) para el partido {row['Equip local']} vs {row['Equip visitant']}: >> ")

        if df_final.loc[idx, 'Categoria'] == "ESPORT ADAPTAT":
            aa2 = input(f"Ingrese el nombre del Árbitro Asistente2 (AA2) para el partido {row['Equip local']} vs {row['Equip visitant']}: >> ")
            an = input(f"Ingrese el nombre del Anotador (AN) para el partido {row['Equip local']} vs {row['Equip visitant']}: >> ")
            cr = input(f"Ingrese el nombre del Cronometrador (CR) para el partido {row['Equip local']} vs {row['Equip visitant']}: >> ")
            rll = input(f"Ingrese el nombre del Reloj de Lanzamiento (RLL) para el partido {row['Equip local']} vs {row['Equip visitant']}: >> ")

        # Asignar los valores ingresados a las columnas correspondientes
        df_final.loc[idx, 'AP'] = ap
        df_final.loc[idx, 'AA'] = aa

        if df_final.loc[idx, 'Categoria'] == "ESPORT ADAPTAT":
            df_final.loc[idx, 'AA2'] = aa2
            df_final.loc[idx, 'AN'] = an
            df_final.loc[idx, 'CR'] = cr
            df_final.loc[idx, 'RLL'] = rll
            df_final.loc[idx, 'CALLER'] = ''



Partido: COMKEDEM vs CORPORACIÓ FISIOGESTIÓN CB ROSER Categoria: ESPORT ADAPTAT


In [28]:
if False:
    import pandas as pd
    import ipywidgets as widgets
    from IPython.display import display

    # Función para ingresar datos de los árbitros
    def ingresar_arbitros(df_final):
        for idx, row in df_final.iterrows():
            print(f"\nPartido: {row['Equip local']} vs {row['Equip visitant']} Categoria: {row['Categoria']}")
            
            # Crear widgets para ingresar los árbitros
            ap_widget = widgets.Text(
                description=f'AP (Idx {idx}):',
                placeholder='Árbitro Principal'
            )
            aa_widget = widgets.Text(
                description=f'AA (Idx {idx}):',
                placeholder='Árbitro Asistente'
            )
            
            # Mostrar los widgets
            display(ap_widget, aa_widget)
            
            # Función para capturar los valores ingresados
            def on_submit(change):
                df_final.loc[idx, 'AP'] = ap_widget.value
                df_final.loc[idx, 'AA'] = aa_widget.value
                print(f"Árbitros registrados para el partido {row['Equip local']} vs {row['Equip visitant']}: AP = {ap_widget.value}, AA = {aa_widget.value}")
            
            # Llamar a la función cuando cambien los valores
            ap_widget.observe(on_submit, 'value')
            aa_widget.observe(on_submit, 'value')

    # Llamar a la función para ingresar los árbitros
    ingresar_arbitros(df_final)

    # Mostrar el DataFrame actualizado (después de la interacción)
    df_final


In [29]:
# Ordenamos por dia de partido
## df_final = df_final.sort_values(by=['Dia de joc'], ascending=[True])
## df_final

In [30]:
 # Cargamos los datos de los informadores de comité
comite = pd.read_pickle("data/2425ComiteArbitral_Informadores.pkl")
comite["NOM COMPLET"] = comite["NOMS"] + " " + comite["COGNOMS"]
comite.head()

,COGNOMS,NOMS,LLICENCIES,CATEGORIA,RRTT,informador,NOM COMPLET
12,ALEMANY MODUBAR,RAQUEL,94,AUX. DE TAULA FEB (G. 1 i 2),Barcelona,SI,RAQUEL ALEMANY MODUBAR
16,ALMAZAN MARCHENA,ANTONIO,426,AUX. DE TAULA ACB / INT.,Barcelona,SI,ANTONIO ALMAZAN MARCHENA
25,ARELLANO MEDINA,DAVID,12313,AUX. DE TAULA FEB (G. 1 i 2),Barcelona,SI,DAVID ARELLANO MEDINA
45,BAÑOS MARTINEZ,ROBERTO,120,AUX. DE TAULA ACB / INT.,Barcelona,SI,ROBERTO BAÑOS MARTINEZ
57,BAYO DE LA CRUZ,MARC,310,AUX. DE TAULA ACB / INT.,Barcelona,SI,MARC BAYO DE LA CRUZ


In [31]:
# Crear un conjunto con los nombres del comité para facilitar la búsqueda
nombres_comite = set(comite['NOM COMPLET'])

In [32]:
# Aplicar la función por filas (axis=1) y añadir la nueva columna "Informador"
df_final['INFORME'] = df_final[['AN', 'CR', 'RLL', 'CALLER']].apply(verificar_informador, axis=1, args=(nombres_comite,))

##### ENVIAR CORREO CON PARTIDOS

In [33]:
# Convertir el DataFrame a formato HTML
all_empty = (df_final['AA2'] == "").all()

# Crear una copia explícita del DataFrame
df_final2 = df_final.copy()

if all_empty:
    df_final2 = df_final2[['INFORME', 'Num.', 'Equip local', 'Equip visitant', 
                        'Dia de joc', 'Camp', 'Competició', 'Categoria', 
                        'AP', 'AA', 'AN', 'CR', 'RLL', 'CALLER']]
    # .to_html(index=False)
    
else:
    df_final2 = df_final2[['INFORME', 'Num.', 'Equip local', 'Equip visitant', 
                        'Dia de joc', 'Camp', 'Competició', 'Categoria', 
                        'AP', 'AA', 'AA2', 'AN', 'CR', 'RLL', 'CALLER']]
    # .to_html(index=False)
    


In [34]:
df_final2.loc[:, 'INFORME'] = df_final2.loc[:, 'INFORME'].apply(transformar_informe)

In [35]:
df_final2

,INFORME,Num.,Equip local,Equip visitant,Dia de joc,Camp,Competició,Categoria,AP,AA,AN,CR,RLL,CALLER
0,,6048,SESE - VITALDENT,TGN BÀSQUET A,08/03/2025 17:45,PAVELLO MUNICIPAL VIRREI AMAT,FASE PRÈVIA,SUPER COPA FEMENINA,RAQUEL GUIL GARCIA,POL MARCET CALDERS,VICTOR GOMEZ PLAZA,SERGI RAMIREZ MITJANS,MIREIA ROBERT MATEO,MARCEL COLOME PORTA
1,,71477,COMKEDEM,CORPORACIÓ FISIOGESTIÓN CB ROSER,09/03/2025 17:00,CEM LLARS MUNDET,BCR LLIGA CATALANA 2,ESPORT ADAPTAT,VICTOR ASENJO CARVAJAL,ARACELI MULET ALLES,MARC ARTIGAS SUAREZ,SERGI RAMIREZ MITJANS,,


In [36]:
# Convertir el DataFrame a formato HTML con resalte condicional y color de fila
columnas_a_usar = ['Num.', 'Equip local', 'Equip visitant', 'Dia de joc', 'Camp', 'Competició', 'Categoria', 'AP', 'AA', 'AN', 'CR', 'RLL', 'CALLER', 'INFORME']

In [37]:
# Convertir el DataFrame a formato HTML con resalte condicional
html_table = "<table border='1'>"
html_table += "<tr>" + "".join([f"<th>{col}</th>" for col in columnas_a_usar]) + "</tr>"

for _, fila in df_final2[columnas_a_usar].iterrows():
    html_table += fila_resaltada(fila, columnas_a_usar)



# for _, fila in df_final2[columnas_a_usar].iterrows():
#     html_table += "<tr>"
#    for columna in columnas_a_usar:
#        valor = fila[columna]
#        if columna in ['AN', 'CR', 'RLL', 'CALLER']:
#            html_table += resaltar_celda(valor)
#        else:
#            html_table += f"<td>{valor}</td>"
#    html_table += "</tr>"

html_table += "</table>"


In [38]:
# Configuración del correo electrónico
remitente = "ramitjans@gmail.com"
destinatario = "sergi9_14@hotmail.com"
asunto = "Designaciones de este fin de semana"
cuerpo_html = f"""
<html>
<head></head>
<body>
    <p>A continuación te adjunto los partidos de la semana:</p>
    {html_table}
    <p>Cobrarás por ello, {round(df_final['precio'].sum(), ndigits=2)} euros </p>
</body>
</html>
"""

# Crear el mensaje de correo
mensaje = MIMEMultipart("alternative")
mensaje['From'] = remitente
mensaje['To'] = destinatario
mensaje['Subject'] = asunto

# Agregar el cuerpo HTML
mensaje.attach(MIMEText(cuerpo_html, 'html'))

# Establecer la conexión con el servidor SMTP
servidor = smtplib.SMTP('smtp.gmail.com', 587)
servidor.starttls()
servidor.login(remitente, "qvhq kxha uiev culc")  # Reemplaza con tu contraseña o usa una App Password

# Enviar el correo
servidor.sendmail(remitente, destinatario, mensaje.as_string())

# Cerrar la conexión con el servidor
servidor.quit()

print("Correo enviado con éxito.")

Correo enviado con éxito.


#### INSERTAR PARTIDOS AL GOOGLE CALENDAR

In [39]:
# Subimos el partido en el google drive
# from datetime import datetime
# from datetime import timedelta
from paquetes.googleCalendarPermisons import conectarCredenciales, generarServicio, listarCalendarios, buscarID
from paquetes.funciones_partidos import tiempoAntes, tiempoPartido, generaEventosCalendar

In [40]:
# Obtenemos las credenciales
creds = conectarCredenciales("token.pickle")

Credenciales refrescadas correctamente.


In [41]:
# Creamos el servicio
servicio = generarServicio(creds)

In [42]:
# listamos todos los calendarios disponibles
listaCalendarios = listarCalendarios(servicio)

In [43]:
# Buscamos el id que necesitamos
idN = buscarID(listaCalendarios, nombreCalendario="FCBQ")

In [44]:
# Función para encontrar en qué columna está el string
def encontrar_columna(row, valor_buscar):
    # Buscar en cada columna si contiene el valor especificado
    for col in df_final2.columns:
        if row[col] == valor_buscar:
            return col
    return None

In [45]:
# Buscar en todas las columnas dónde aparece "SERGI RAMIREZ"
## nombre_buscado = "SERGI RAMIREZ MITJANS"
df_final2.loc[:, 'posicion'] = df_final2.apply(encontrar_columna, axis = 1, valor_buscar = "SERGI RAMIREZ MITJANS")

In [46]:
# Convertimos el dia de juego en datetime
df_final2.loc[:, 'Dia de joc'] = pd.to_datetime(df_final2.loc[:, 'Dia de joc'], format='%d/%m/%Y %H:%M')

In [47]:
# Creamos dos columnas nuevas
df_final2.loc[:, "tiempo antes"] = df_final2.loc[:, "Dia de joc"]
df_final2.loc[:, "tiempo partido"] = df_final2.loc[:, "Dia de joc"]
df_final2.loc[:, "urlPartido"] = df_final.loc[:, "urlPartido"]
df_final2.loc[:, "A cobrar"] = df_final.loc[:, "A cobrar"]

In [48]:
# Calculamos el tiempo antes necesario para llegar a pista
df_final2 = tiempoAntes(df_final2)

# Calculamos el tiempo de duración del partido
df_final2 = tiempoPartido(df_final2)

In [49]:
# Creamos los eventos y los subimos
generaEventosCalendar(df_final2, servicio, idN)

Created event: oftap0ld27stioot3b2dar9t8k
Created event: 17orcok15d73s8i3d277fba7o4
